In [1]:
using JuMP
using Base.Test
const MOI = MathOptInterface

INFO: Recompiling stale cache file /home/blegat/.julia/lib/v0.6/JuMP.ji for module JuMP.


MathOptInterface

In [2]:
module StructJuMP

export StructuredModel

using JuMP # To reexport, should be using (not import)

mutable struct StructuredModel <: JuMP.AbstractModel
    variables::Vector{JuMP.AbstractVariable}
    varnames::Vector{String}
    constraints::Vector{JuMP.AbstractConstraint}
    
    objdict::Dict{Symbol, Any}

    function StructuredModel()
        new(JuMP.AbstractVariable[], String[], JuMP.AbstractConstraint[], Dict{Symbol, Any}())
    end
end

JuMP.object_dictionary(m::StructuredModel) = m.objdict

struct StructuredVariable <: JuMP.AbstractVariableRef
    model::StructuredModel # `model` owning the variable
    idx::Int               # Index in `model.variables`
end
Base.copy(v::StructuredVariable) = v
JuMP.name(v::StructuredVariable) = v.model.varnames[v.idx]
JuMP.variabletype(::StructuredModel) = StructuredVariable

function JuMP.addvariable(m::StructuredModel, v::JuMP.AbstractVariable, name::String="")
    push!(m.variables, v)
    push!(m.varnames, name)
    StructuredVariable(m, length(m.variables))
end

struct StructuredConstraintRef
    model::StructuredModel # `model` owning the constraint
    idx::Int               # Index in `model.constraints`
end

function JuMP.addconstraint(m::StructuredModel, c::JuMP.AbstractConstraint, name::String="")
    push!(m.constraints, c)
    StructuredConstraintRef(m, length(m.constraints))
end

end

StructJuMP

In [21]:
m = StructJuMP.StructuredModel()

StructJuMP.StructuredModel(JuMP.AbstractVariable[], String[], JuMP.AbstractConstraint[], Dict{Symbol,Any}())

In [22]:
@variable m x
@test x isa StructJuMP.StructuredVariable
@test string(x) == "x"
@test m[:x] === x

Test Passed

In [23]:
@variable m y[1:2, 1:2]
@test y isa Matrix{StructJuMP.StructuredVariable}
@test string(y[1, 1]) == "y[1,1]"

Test Passed

In [25]:
cref = @constraint m x in MOI.LessThan(1.0)
c = m.constraints[cref.idx]
@test m.constraints[cref.idx] isa JuMP.SingleVariableConstraint{StructJuMP.StructuredVariable}

Test Passed

In [7]:
@constraint m 1 <= x <= 2;

In [8]:
@constraint m 2x+1 == 2;

In [9]:
@constraint m 1 <= 2x+1 <= 2;

In [10]:
@constraint m 2x^2+1 == 2;

In [11]:
@constraint m 1 <= 2x^2+1 <= 2;

LoadError: [91mIn @constraint(m,1 <= 2 * x ^ 2 + 1 <= 2): Two-sided quadratic constraints not supported. (Try @NLconstraint instead.)[39m

In [12]:
@constraint m [x, x] .== [1, 2];

In [13]:
@constraint m [2x, x+1] .== [1, 2];

In [14]:
@constraint m [1, 2] .<= [2x, x+1] .<= [3, 4];

In [15]:
@variable m y[1:2, 1:2]

LoadError: [91mA variable or constraint named y is already attached to this model. If creating variables programmatically, use the anonymous variable syntax x = @variable(m, [1:N], ...).[39m

In [16]:
2x + 1

2 x + 1

In [17]:
x^2 + 1

x² + 1

In [18]:
m = JuMP.Model()

A JuMP Model

In [19]:
@variable m z

z

In [20]:
@constraint m z == 1

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}}(1))